In [1]:
# ==============================================================
# 📊 PIPELINE COMPLETO: Evaluación de "memory quality" en logs LLM
# ==============================================================

import pandas as pd
import json
import re
import numpy as np
from collections import Counter
import itertools



In [2]:
# ==============================================================
# 1️⃣ CARGA Y FILTRADO DE DATOS
# ==============================================================


file_path = "log_llm_prompts.csv"   
df = pd.read_csv(file_path)
df = df[df["ref"] == "facts"].copy()

print(f"✅ Total de registros con ref='facts': {len(df)}")

✅ Total de registros con ref='facts': 270


In [5]:
# ==============================================================
# 2️⃣ LIMPIEZA Y PARSEO DE RESPUESTAS JSON
# ==============================================================

def extract_facts(response_text):
    """Extrae lista de 'facts' de la respuesta JSON del modelo."""
    try:
        # Quitar etiquetas tipo ```json
        clean_text = re.sub(r"```json|```", "", str(response_text)).strip()
        data = json.loads(clean_text)
        if "facts" in data and isinstance(data["facts"], list):
            return data["facts"]
        return None
    except Exception:
        return None

df["facts_parsed"] = df["response"].apply(extract_facts)
df["facts_count"] = df["facts_parsed"].apply(lambda x: len(x) if isinstance(x, list) else 0)
df.head(10)


,id,uuid,ref,version,prompt,response,created_at,facts_parsed,facts_count
0,1,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.5,"\nEres un extractor de ""hechos duraderos"". Dev...","{\n ""facts"": [\n { ""key"": ""estado_animo_ho...",2025-10-14 18:55:58,"[{'key': 'estado_animo_hoy', 'value': 'un poco...",1
1,2,3175f824-d9de-4725-b8a7-eebfefb2bf70,facts,v3.3.5,"\nEres un extractor de ""hechos duraderos"". Dev...","```json\n{\n ""facts"": []\n}\n```",2025-10-14 20:49:01,[],0
4,5,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.5,"\nEres un extractor de ""hechos duraderos"". Dev...","{\n ""facts"": []\n}",2025-10-15 11:35:34,[],0
6,7,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.5,"\nEres un extractor de ""hechos duraderos"". Dev...","```json\n{\n ""facts"": []\n}\n```",2025-10-15 11:36:13,[],0
10,11,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.6,"\nEres un extractor de ""hechos duraderos"". Dev...","{\n ""facts"": []\n}",2025-10-15 11:41:17,[],0
13,14,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.6,"\nEres un extractor de ""hechos duraderos"". Dev...","{\n ""facts"": [\n { ""key"": ""nombre"", ""value...",2025-10-15 11:42:01,"[{'key': 'nombre', 'value': 'Guillermo'}]",1
14,15,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.6,"\nEres un extractor de ""hechos duraderos"". Dev...","```json\n{\n ""facts"": []\n}\n```",2025-10-15 11:47:15,[],0
18,19,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.6,"\nEres un extractor de ""hechos duraderos"". Dev...","{\n ""facts"": [\n { ""key"": ""nombre"", ""value...",2025-10-15 11:47:43,"[{'key': 'nombre', 'value': 'Guillermo'}]",1
22,23,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.6,"\nEres un extractor de ""hechos duraderos"". Dev...","{\n ""facts"": [\n { ""key"": ""nombre"", ""value...",2025-10-15 11:48:04,"[{'key': 'nombre', 'value': 'billar'}, {'key':...",3
24,25,6ac3190b-faf2-4eed-b6a5-9a81cceb6df0,facts,v3.3.6,"\nEres un extractor de ""hechos duraderos"". Dev...","{\n ""facts"": [\n { ""key"": ""nombre"", ""value...",2025-10-15 11:48:46,"[{'key': 'nombre', 'value': 'Guillermo'}]",1


In [6]:
# ==============================================================
# 3️⃣ FUNCIONES DE MÉTRICAS
# ==============================================================

try:
    from sentence_transformers import SentenceTransformer, util
    model_path = r"paraphrase-multilingual-MiniLM-L12-v2"
    model = SentenceTransformer(model_path)
    USE_EMBEDDINGS = True
    print("✅ Usando SentenceTransformer para similitud semántica.")
except Exception:
    USE_EMBEDDINGS = False
    print("⚠️ No se encontró 'sentence-transformers'. Se usará similitud de texto básica.")




✅ Usando SentenceTransformer para similitud semántica.


In [7]:
# ---- Métricas ----

def form_validity(df):
    """Porcentaje de respuestas con JSON válido y campos key/value completos."""
    valid = df["facts_parsed"].apply(
        lambda x: isinstance(x, list) and all("key" in f and "value" in f for f in x)
    )
    return valid.mean()


def recall_estimate(df):
    """Proporción de prompts con al menos un fact detectado."""
    return (df["facts_count"] > 0).mean()


def semantic_consistency_by_thread(df):
    """
    Calcula consistencia considerando cada hilo (uuid) por separado.
    Así evitamos contar como contradicciones los cambios entre versiones distintas.
    """
    thread_scores = {}

    for thread, subdf in df.groupby("uuid"):
        all_facts = [f for sublist in subdf["facts_parsed"].dropna() for f in sublist if isinstance(f, dict)]
        key_groups = {}
        for f in all_facts:
            key = f["key"]
            val = f["value"]
            key_groups.setdefault(key, set()).add(val)
        if not key_groups:
            thread_scores[thread] = np.nan
            continue
        contradictions = sum(len(v) > 1 for v in key_groups.values())
        thread_scores[thread] = 1 - (contradictions / len(key_groups))

    # Consistencia promedio entre hilos (ponderada)
    return np.nanmean(list(thread_scores.values())), thread_scores


def simple_text_similarity(a, b):
    """Similitud básica si no hay embeddings disponibles."""
    a, b = a.lower(), b.lower()
    inter = len(set(a.split()) & set(b.split()))
    union = len(set(a.split()) | set(b.split()))
    return inter / union if union > 0 else 0


def fidelity_score(df):
    """Mide similitud semántica entre los values y el texto del prompt."""
    sims = []
    for _, row in df.iterrows():
        prompt = str(row["prompt"])
        facts = row["facts_parsed"]
        if isinstance(facts, list):
            for f in facts:
                val = str(f.get("value", ""))
                if val:
                    try:
                        if USE_EMBEDDINGS:
                            sim = util.cos_sim(model.encode(prompt), model.encode(val))[0][0].item()
                        else:
                            sim = simple_text_similarity(prompt, val)
                        sims.append(sim)
                    except Exception:
                        continue
    return np.mean(sims) if sims else 0


def memory_quality_score(df):
    """Combina todas las métricas en una sola puntuación global."""
    form = form_validity(df)
    recall = recall_estimate(df)
    consistency, _ = semantic_consistency_by_thread(df)   # ✅ corregido
    fidelity = fidelity_score(df)
    score = np.mean([form, recall, consistency, fidelity])
    return {
        "Form_Validity": round(form, 3),
        "Recall": round(recall, 3),
        "Consistency": round(consistency, 3),
        "Fidelity": round(fidelity, 3),
        "Memory_Quality_Score": round(score, 3)
    }

# 🧠 Evaluación de la calidad de memoria factual del modelo

Estas métricas cuantifican **qué tan bien el modelo guarda y mantiene información factual** (pares *key–value*) extraída de los prompts de los usuarios.  
Todas se expresan entre **0 y 1**, donde valores más altos indican mejor desempeño.

---

## 🧩 1. Form_Validity — *Validez estructural*

**Qué mide:**  
Si las respuestas están correctamente formateadas en JSON y contienen los campos esperados (`"key"` y `"value"`).

**Cómo se calcula:**  
Proporción de respuestas donde el JSON es válido y cada fact tiene `key` y `value`.

**Interpretación:**  
- `1.0` → todas las respuestas están bien estructuradas.  
- `< 1.0` → hay errores de formato o campos faltantes.

**Aporta:**  
Detecta fallos técnicos o de formato antes de analizar el contenido semántico.

---

## 🔎 2. Recall — *Cobertura o detección de hechos*

**Qué mide:**  
Cuántos prompts generan al menos un hecho (`facts_count > 0`).

**Cómo se calcula:**  
Número de respuestas con facts distintos de vacío dividido entre el total de respuestas.

**Interpretación:**  
- Valores altos → el modelo detecta hechos con frecuencia.  
- Valores bajos → omite información factual.

**Aporta:**  
Evalúa la **capacidad de detección** del modelo.

---

## ⚖️ 3. Consistency — *Coherencia semántica*

**Qué mide:**  
Si una misma *key* mantiene el mismo *value* dentro de un hilo (`uuid`) o cambia contradictoriamente.

**Cómo se calcula:**  
$$
\text{Consistency} = 1 - \frac{\text{#keys con valores diferentes}}{\text{#keys totales}}
$$

**Interpretación:**  
- `1.0` → coherente (sin contradicciones).  
- `0.0` → cada key tiene varios valores.

**Aporta:**  
Mide la **estabilidad de la memoria factual** del modelo.

---

## 💬 4. Fidelity — *Fidelidad contextual*

**Qué mide:**  
Si los *values* almacenados provienen realmente del texto del prompt.

**Cómo se calcula:**  
Similitud semántica entre prompt y *value*:  
- Con *embeddings* (`SentenceTransformer`): similitud coseno (0–1).  
- Sin embeddings: coincidencia de palabras (Jaccard).

**Interpretación:**  
- `> 0.8` → facts fieles al texto.  
- `< 0.5` → facts imprecisos o inventados.

**Aporta:**  
Evalúa la **precisión semántica** del modelo.

---

## 🧮 5. Memory_Quality_Score — *Puntuación global de memoria*

**Qué mide:**  
Resumen global del desempeño combinando las cuatro métricas.

**Cómo se calcula:**  
$$
\text{Memory\_Quality\_Score} = \frac{\text{Form\_Validity} + \text{Recall} + \text{Consistency} + \text{Fidelity}}{4}
$$

**Interpretación:**

| Rango | Significado |
|--------|--------------|
| 0.8 – 1.0 | Excelente — memoria sólida y precisa. |
| 0.6 – 0.8 | Buena — estructura correcta, pero con mejoras posibles. |
| 0.4 – 0.6 | Media — memoria parcial o inconsistente. |
| < 0.4 | Débil — no guarda hechos de forma fiable. |

**Aporta:**  
Permite comparar versiones o configuraciones del modelo con una única puntuación estándar.

---

## 📊 Resumen general de las métricas

| Métrica | Qué mide | Rango ideal | Aporta |
|----------|-----------|--------------|--------|
| **Form_Validity** | Corrección estructural del JSON | ≥ 0.98 | Calidad de formato. |
| **Recall** | Detección de hechos en los prompts | ≥ 0.5 | Capacidad de extracción. |
| **Consistency** | Coherencia de valores por key | ≥ 0.8 | Estabilidad de memoria. |
| **Fidelity** | Fidelidad semántica prompt–value | ≥ 0.75 | Precisión de contenido. |
| **Memory_Quality_Score** | Promedio global | ≥ 0.7 | Calidad total de memoria. |


In [8]:
# ==============================================================
# 4️⃣ EJECUCIÓN DE EVALUACIÓN
# ==============================================================

results = memory_quality_score(df)

print("\n📈 Resultados de evaluación del modelo:")
for k, v in results.items():
    print(f"{k:25s}: {v}")

# ==============================================================
# 5️⃣ INSPECCIÓN DE CONSISTENCIA Y CONTRADICCIONES
# ==============================================================

def contradiction_report_by_thread(df):
    """Devuelve contradicciones detectadas dentro de cada hilo (uuid)."""
    report = {}
    for thread, subdf in df.groupby("uuid"):
        all_facts = [f for sublist in subdf["facts_parsed"].dropna() for f in sublist if isinstance(f, dict)]
        key_groups = {}
        for f in all_facts:
            key_groups.setdefault(f["key"], set()).add(f["value"])
        contradictions = {k: list(v) for k, v in key_groups.items() if len(v) > 1}
        report[thread] = contradictions
    return report


consistency_global, consistency_by_thread = semantic_consistency_by_thread(df)
print(f"\nConsistencia global (media): {consistency_global:.3f}")
print("Consistencia por hilo (uuid):")
for t, s in consistency_by_thread.items():
    print(f"  {t}: {s:.3f}")

report = contradiction_report_by_thread(df)
for t, contradictions in report.items():
    if contradictions:
        print(f"\n🧩 {t} — Contradicciones detectadas:")
        for k, vals in list(contradictions.items())[:5]:
            print(f"  - {k}: {vals}")




📈 Resultados de evaluación del modelo:
Form_Validity            : 1.0
Recall                   : 0.207
Consistency              : 0.743
Fidelity                 : 0.127
Memory_Quality_Score     : 0.519

Consistencia global (media): 0.743
Consistencia por hilo (uuid):
  0caf55d1-0796-4d6f-9c96-425d0b2347ed: 0.800
  10f5b2e5-aadc-40a5-9692-3aa601fd296d: nan
  240606c4-db5c-41e7-826b-f5d6ca84fb2d: nan
  2e9c9bcc-fa40-436c-b9da-d4a910e904d9: nan
  3175f824-d9de-4725-b8a7-eebfefb2bf70: nan
  44946cfa-8d89-4f5d-a0b6-974733fa01c4: 0.400
  5255ae6c-ab8a-4d55-974b-7607b2b34eda: nan
  66ec692c-4ef2-4b1e-b144-3f141c898978: nan
  6ac3190b-faf2-4eed-b6a5-9a81cceb6df0: 0.500
  8db25c30-4a6d-4673-842d-79cb2942acb2: nan
  976b4fee-8cf4-4305-9690-0c83a170ac9a: 0.800
  c9f921b2-b3c4-4a96-ab97-d0050efc5e73: 0.700
  cdbb3877-7819-45fb-bfed-10b34d8dfc1c: 1.000
  df5645d1-3be6-4444-b051-7d47b706fa79: nan
  f3c64f74-07d5-4a3d-b7d7-fe1a9e9e0be9: nan
  fb13f0d9-3e09-43de-a5cc-22b5f8923fe0: 1.000

🧩 0caf55d1-0

# 📊 Interpretación de resultados de evaluación del modelo

A continuación se explica el significado de los resultados obtenidos al evaluar la calidad de memoria factual del modelo.

---

## 📈 Resumen general de métricas

| Métrica | Valor | Interpretación |
|----------|--------|----------------|
| **Form_Validity** | **1.0** | El 100 % de las respuestas están correctamente estructuradas en formato JSON con los campos `key` y `value`. El modelo maneja bien la forma técnica de almacenamiento. |
| **Recall** | **0.207** | Solo el **20.7 %** de los prompts generan algún hecho guardado. El modelo detecta hechos pocas veces: en casi 8 de cada 10 casos no almacena nada. |
| **Consistency** | **0.743** | Nivel de coherencia medio–alto. En promedio, un 74 % de las *keys* mantienen valores estables dentro de cada hilo. El resto presentan contradicciones. |
| **Fidelity** | **0.127** | Fidelidad semántica muy baja. Los *values* guardados no coinciden bien con el contenido real del prompt. El modelo extrae hechos válidos pero con información poco fiel. |
| **Memory_Quality_Score** | **0.519** | Puntuación global media. La memoria factual funciona, pero su desempeño es limitado: estructura sólida, coherencia parcial y baja fidelidad contextual. |

---

## 🧠 Interpretación general

- El modelo **estructura correctamente** los hechos (`Form_Validity = 1.0`), lo que indica una salida limpia y sin errores de formato.  
- Sin embargo, **detecta pocos hechos** (`Recall = 0.207`), lo que revela una baja sensibilidad al identificar información relevante.  
- La **coherencia interna** es aceptable (`Consistency = 0.743`): suele mantener la misma información dentro de un mismo hilo.  
- La **fidelidad semántica** es muy baja (`Fidelity = 0.127`): los valores guardados no siempre reflejan lo que el usuario dijo.  
- En conjunto, el modelo **recuerda bien la estructura pero no siempre el contenido correcto**.

---

## ⚖️ Consistencia global y por hilo (uuid)

- **Consistencia global media:** `0.743`  
  Indica estabilidad general moderada.  
- **Consistencia por hilo:** varía entre `0.4` y `1.0`, mostrando que la estabilidad depende de la conversación concreta.  
  Los valores `nan` corresponden a hilos sin hechos suficientes para evaluar.

---

## 🧩 Contradicciones detectadas por hilo

### 🧠 `0caf55d1-0796-4d6f-9c96-425d0b2347ed`
**Contradicción:**  
- `estado_trabajo_actual`: `["trabajando", "trabajando con un proyecto de inteligencia artificial"]`  
**Interpretación:**  
Actualización o ampliación del valor, no una contradicción grave.  
🟢 *Contradicción leve.*

---

### 🧠 `44946cfa-8d89-4f5d-a0b6-974733fa01c4`
**Contradicciones:**  
- `nombre`: `["Yeray", "Eugenio", "Mario"]` → contradicción fuerte.  
- `gustos_constantes`: `["melón, fresa", "ya no me gusta la uva"]` → se niega un gusto anterior.  
- `estado_animo_hoy`: `["contento", "enfadado"]` → variación emocional esperable.  
🔴 *Contradicciones severas en identidad y gusto.*

---

### 🧠 `6ac3190b-faf2-4eed-b6a5-9a81cceb6df0`
**Contradicciones:**  
- `estado_animo_hoy`: “un poco cansado”, “un poquito mejor”, “muy bien”.  
- `nombre`: “Guillermo”, “billar” (error semántico).  
- `intereses_duraderos`: “aprendizaje”, “inteligente”.  
- `musica_preferida`: “Misfits”, “Megadeth”, “Metallica”.  
🟠 *Contradicciones moderadas, mezcla de categorías y variabilidad emocional.*

---

### 🧠 `976b4fee-8cf4-4305-9690-0c83a170ac9a`
**Contradicción:**  
- `fecha_nacimiento_o_edad`: `["1973", "30"]` → fecha vs edad numérica.  
🔵 *Contradicción leve por tipo de dato.*

---

### 🧠 `c9f921b2-b3c4-4a96-ab97-d0050efc5e73`
**Contradicciones:**  
- `gustos_constantes`: “notas de la película”, “libros”.  
- `libros_preferidos`: múltiples títulos distintos (no necesariamente error).  
- `lectura_actual`: “libro”, “mitad del libro”.  
🟡 *Contradicciones leves o reflejo de progreso en la lectura.*

---

## 📘 Conclusión

- El modelo **guarda correctamente la estructura** de los hechos (`Form_Validity = 1.0`).  
- **Detecta pocos hechos** (`Recall = 0.207`), lo que limita su cobertura.  
- **Mantiene cierta coherencia interna** (`Consistency = 0.743`), aunque con contradicciones en identidad o gustos.  
- **Baja fidelidad semántica** (`Fidelity = 0.127`): los valores almacenados no reflejan con precisión el texto del usuario.  
- **Memoria global moderada** (`Memory_Quality_Score = 0.519`): el modelo tiene una base técnica sólida pero carece de precisión semántica.

> En resumen, el modelo **recuerda la forma, pero no siempre el fondo**:  
> guarda los hechos de manera estructurada, pero no necesariamente la información correcta o coherente con lo que el usuario expresó.


# Importante a tener en cuenta

En la metrica de consistencia semantica penaliza cuando hay valores diferentes en los *values*, por lo tanto una correción por equivocación del usuario podria llegar a perjudicar a dicha metrica en ese aspecto. Ademas, es posible que el usuario este actualizando la información y se penalize en la métrica. 
Se podria cambiar el codigo al siguiente:



In [ ]:
def semantic_consistency_by_thread(df):
    """
    Calcula consistencia por hilo (uuid), ignorando correcciones válidas,
    cambios leves o keys naturalmente variables.
    """
    # Configuraciones básicas
    dynamic_keys = {"estado_animo_hoy", "actividad_actual", "estado_trabajo_actual"}
    correction_words = ["me equivoqué", "en realidad", "rectifico", "corregir", "quise decir"]
    thread_scores = {}

    # Iterar por hilo
    for thread, subdf in df.groupby("uuid"):
        all_facts = [f for sublist in subdf["facts_parsed"].dropna() for f in sublist if isinstance(f, dict)]
        key_groups = {}
        for f in all_facts:
            key = f["key"]
            val = f["value"]
            key_groups.setdefault(key, set()).add(val)

        if not key_groups:
            thread_scores[thread] = np.nan
            continue

        contradictions = 0
        for key, values in key_groups.items():
            if len(values) > 1:
                # 1️⃣ No penalizar keys dinámicas
                if key in dynamic_keys:
                    continue

                values_list = list(values)
                is_contradiction = True

                # 2️⃣ Comprobar similitud semántica entre values
                for i in range(len(values_list)):
                    for j in range(i + 1, len(values_list)):
                        val1, val2 = values_list[i], values_list[j]
                        if USE_EMBEDDINGS:
                            sim = util.cos_sim(model.encode(val1), model.encode(val2))[0][0].item()
                        else:
                            sim = simple_text_similarity(val1, val2)
                        if sim > 0.8:  # son casi iguales
                            is_contradiction = False

                # 3️⃣ Comprobar si el prompt tiene palabras de corrección
                prompts = " ".join(subdf["prompt"].astype(str)).lower()
                if any(w in prompts for w in correction_words):
                    is_contradiction = False

                if is_contradiction:
                    contradictions += 1

        thread_scores[thread] = 1 - (contradictions / len(key_groups))

    # Promedio global
    return np.nanmean(list(thread_scores.values())), thread_scores


| Tipo de cambio                                   | Resultado                               |
| ------------------------------------------------ | --------------------------------------- |
| “Guillermo” vs “Guille”                          | No se penaliza (similitud alta).        |
| “Me equivoqué, mi nombre es Mario”               | No se penaliza (palabra de corrección). |
| `estado_animo_hoy`: “contento” → “cansado”       | No se penaliza (key dinámica).          |
| “Guillermo” → “Pedro” sin contexto de corrección | Penalización (contradicción real).      |


Después de aplicar esta versión:

- La consistencia sube ligeramente (porque ya no penaliza cambios legítimos).

- El análisis de contradicciones se vuelve más semántico y realista.

- La métrica se vuelve útil para medir errores verdaderos de memoria factual, no simples actualizaciones